In [3]:
%load_ext sql

In [4]:
%sql sqlite:///library.db

'Connected: @library.db'

In [5]:
%%sql 

CREATE TABLE PhysicalLibraryItems(
  	AssetTag int,
    Status varchar(255) NOT NULL,
    ShelfNumber float DEFAULT NULL, 
  	PRIMARY KEY (AssetTag)
);

CREATE TABLE Publications(
	AssetTag int,
	Status varchar(255),
    ShelfNumber float,
    ISBN char(13),
    Title varchar(255) NOT NULL,
    Genre varchar(255) NOT NULL,
    Author varchar(255) NOT NULL,
    Subject varchar(255),
    Edition int CHECK (Edition > 0),
    Type varchar(255) NOT NULL,
    ContentRating varchar(255),
    Publisher varchar(255) NOT NULL,
	DateOfPublication DATE DEFAULT '0000-00-00',
    PRIMARY KEY (AssetTag, ISBN),
    FOREIGN KEY (AssetTag) REFERENCES PhysicalLibraryItems(AssetTag)
  		ON UPDATE CASCADE
  		ON DELETE CASCADE,
    FOREIGN KEY (Status) REFERENCES PhysicalLibraryItems(Status)
  		ON UPDATE CASCADE
  		ON DELETE CASCADE,
    FOREIGN KEY (ShelfNumber) REFERENCES PhysicalLibraryItems(ShelfNumber)
  		ON UPDATE CASCADE
  		ON DELETE CASCADE
);

CREATE TABLE Media(
	AssetTag int,
	Status varchar(255),
	ShelfNumber float, 
	Title varchar(255),
	YearReleased CHAR(4),
	Genre varchar(255) NOT NULL,
	ProductionStudio varchar(255) NOT NULL,
	Artist varchar(255),
	Length int CHECK (Length > 0),
	NumTracks int CHECK (NumTracks > 0),
	Rating varchar(255) NOT NULL,
	Type varchar(255),
	PRIMARY KEY (AssetTag, Type),
	FOREIGN KEY (AssetTag) REFERENCES PhysicalLibraryItems(AssetTag)
		ON UPDATE CASCADE
		ON DELETE CASCADE,
	FOREIGN KEY (Status) REFERENCES PhysicalLibraryItems(Status)
		ON UPDATE CASCADE
		ON DELETE CASCADE,
	FOREIGN KEY (ShelfNumber) REFERENCES PhysicalLibraryItems(ShelfNumber)
		ON UPDATE CASCADE
		ON DELETE CASCADE
);

CREATE TABLE OnlineBooks(
	ISBN char(13),
  	title varchar(255) NOT NULL,
  	DateOfPublication DATE DEFAULT '0000-00-00', 
  	Genre varchar(255) NOT NULL,
  	Link varchar(255) CHECK (Link LIKE 'http://%'),
  	Publisher varchar(255) NOT NULL,
  	Author varchar(255) NOT NULL,
  	PRIMARY KEY (ISBN)
);

CREATE TABLE LibraryMembers(
	CardNumber int, 
    FirstName varchar(255),
    LastName varchar(255),
    Address varchar(255) NOT NULL,
    Email varchar(255) UNIQUE CHECK (Email LIKE '%@%%'),
    DateOfBirth DATE DEFAULT '0000-00-00',
    PhoneNumber varchar(20) NOT NULL,
    Volunteer bool DEFAULT FALSE,
	Primary Key (CardNumber, FirstName, LastName)
);

CREATE TABLE LibraryEvents(
	EventName varchar(255),
    DateOfEvent DATE DEFAULT '0000-00-00', 
	Fee float CHECK (Fee >= 0), 
    Location varchar(255) NOT NULL,
    RecommendedAge int CHECK (RecommendedAge > 0),
    Type varchar(255) NOT NULL,
    Primary Key (EventName, DateOfEvent)
);

CREATE TABLE LibraryEmployees(
	SIN char(9),
  	FirstName varchar(255) NOT NULL,
  	LastName varchar(255) NOT NULL,
  	Address varchar(255) NOT NULL,
  	Email varchar(255) UNIQUE CHECK (Email LIKE '%@%'),
  	DateOfBirth DATE DEFAULT '0000-00-00',
  	Salary float CHECK (Salary >= 0),
  	PhoneNumber varchar(20) NOT NULL,
  	PRIMARY KEY (SIN)
);

CREATE TABLE FutureItems(
	ItemName varchar(255),
    Type varchar(255) NOT NULL,
    PRIMARY KEY (ItemName)
);

CREATE TABLE BorrowedBy(
	AssetTag int,
	CardNumber int,
	DateBorrowed DATE DEFAULT CURRENT_TIMESTAMP,
    DateDue DATE DEFAULT '0000-00-00',
    DateReturned DATE DEFAULT '0000-00-00',
  	PRIMARY KEY (AssetTag,CardNumber),
  	FOREIGN KEY(AssetTag) REFERENCES PhysicalLibraryItem(AssetTag)
  		ON UPDATE CASCADE
  		ON DELETE CASCADE,
    FOREIGN KEY(CardNumber) REFERENCES LibraryMembers(CardNumber)
  		ON UPDATE CASCADE
  		ON DELETE CASCADE
);

CREATE TABLE DonatedBy(
  	CardNumber int,
	DonerFirstName varchar(255),
    DonerLastName varchar(255), 
    ItemName varchar(255) NOT NULL, 
  	PRIMARY KEY (CardNumber, DonerFirstName, DonerLastName)
  	FOREIGN KEY(CardNumber) REFERENCES LibraryMembers(AssetTag)
  		ON UPDATE CASCADE
  		ON DELETE CASCADE,
    FOREIGN KEY(DonerFirstName) REFERENCES LibraryMembers(FirstName)
  		ON UPDATE CASCADE
  		ON DELETE CASCADE,
    FOREIGN KEY(DonerLastName) REFERENCES LibraryMembers(LastName)
      	ON UPDATE CASCADE
  			ON DELETE CASCADE
);

CREATE TABLE Attending(
 	EventName varchar(255),
    DateOfEvent DATE DEFAULT '0000-00-00',
    CardNumber int,
    FirstName varchar(255) NOT NULL,
    LastName varchar(255) NOT NULL,
    Primary Key(EventName, DateOfEvent, CardNumber)
  	FOREIGN KEY(CardNumber) REFERENCES LibraryMembers(CardNumber)
  		ON UPDATE CASCADE
  		ON DELETE CASCADE,
  	FOREIGN KEY(EventName) REFERENCES LibraryEvents(EventName)
  		ON UPDATE CASCADE
  		ON DELETE CASCADE,
  	FOREIGN KEY(DateOfEvent) REFERENCES LibraryEvents(DateOfEvent)
  		ON UPDATE CASCADE
  		ON DELETE CASCADE
);

CREATE TABLE Working(
	EventName varchar(255),
    DateOfEvent DATE DEFAULT '0000-00-00',
  	SIN char(9),
  	FirstName varchar(255) NOT NULL,
  	LastName varchar(255) NOT NULL,
  	Primary Key(EventName, DateOfEvent, SIN),
  	FOREIGN KEY(EventName) REFERENCES LibraryEvents(EventName)
  		ON UPDATE CASCADE
  		ON DELETE CASCADE,
  	FOREIGN KEY(DateOfEvent) REFERENCES LibraryEvents(DateOfEvent)
  		ON UPDATE CASCADE
  		ON DELETE CASCADE,
  	FOREIGN KEY(SIN) REFERENCES LibraryEmployees(SIN)
  		ON UPDATE CASCADE
  		ON DELETE CASCADE
);

CREATE TABLE DueFines(
	CardNumber int,
    FirstName varchar(255) NOT NULL,
    LastName varchar(255) NOT NULL,
    AssetTag int,
    Amount float CHECK (Amount > 0),
    PRIMARY KEY(CardNumber, AssetTag),
    FOREIGN KEY(CardNumber) REFERENCES LibraryMembers(CardNumber)
  		ON UPDATE CASCADE
  		ON DELETE CASCADE,
	FOREIGN KEY(AssetTag) REFERENCES PhysicalLibraryItems(AssetTag)
  		ON UPDATE CASCADE
  		ON DELETE CASCADE
);

CREATE TRIGGER SetDueDate
AFTER INSERT ON BorrowedBy
BEGIN
	UPDATE BorrowedBy
  	SET DateDue = DateDue + 14
  	WHERE DateBorrowed = GETDATE();
END;

CREATE TRIGGER ItemReturned
AFTER UPDATE ON BorrowedBy
WHEN((OLD.Datedue - OLD.DateBorrowed) > 14)
	BEGIN
    UPDATE DueFines
    SET amount =  CAST((OLD.Datedue - OLD.DateBorrowed) AS FLOAT)
    WHERE OLD.CardNumber = NEW.CardNumber AND OLD.AssetTag = NEW.AssetTag;
END;

* sqlite:///library.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]